# 데이터 불러오기

In [ ]:
import glob
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
txt_file_path = ('/content/drive/MyDrive/Ex6')
txt_list = glob.glob(txt_file_path + "/*.txt")

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 228280
Examples:
 ['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.']


# 데이터 전처리

In [ ]:
import re
import numpy as np
import tensorflow as tf

In [ ]:
def preprocess_sentence(sentence): 
    # 입력된 문장을
    sentence = sentence.lower().strip() #                     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #     2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'(" ")+', " ", sentence) #             3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z.!,¿]+", " ", sentence) #    4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() #                             5. 다시 양쪽 공백을 지웁니다
    sentence = "<start> " + sentence + " <end>" #             6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 : continue # 문장의 길이가 0인 경우 건너뜀
    if sentence[-1] == ':': continue # 문장의 마지막이 ':'으로 끝날 경우 건너뜀 -> 화자가 표기된 의미없는 문장이기 때문
        
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue # 단어별로 토큰화 되기 때문에 15단어를 넘어가는 문장을 제외 -> 토큰 사이즈를 제한
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> is t a verdict <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>',
 '<start> we are accounted poor citizens , the patricians good . <end>']

In [ ]:
def tokenize(corpus): # 토큰화 함수

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 7000단어를 기억할 수 있는 tokenizer 생성 
        filters=' ',  # 문장 정제를 완료했기에 필터 필요치 않음
        oov_token="<unk>"  # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 15로 제한했기에 시쿼스 길이가 15 보다 긴 입력 데이터는 없음
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
        tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor.shape)

[[  2 162  22 ...   0   0   0]
 [  2 359   4 ...   0   0   0]
 [  2   7  68 ...   0   0   0]
 ...
 [  2   5  80 ...   3   0   0]
 [  2  31   5 ...   0   0   0]
 [  2   1  34 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f28c054fbd0>
(179960, 15)


In [ ]:
src_input = tensor[:, :-1] #마지막 토큰을 제외하고 학습 데이터(X)로 저장
tgt_input = tensor[:, 1:] #시작 토큰을 제외하고 타겟 데이터(y)으로 저장

In [ ]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=29)
# sklearn 라이브러리를 활용하여 데이터를 train과 validation으로 나눠줌

In [ ]:
# 모델을 피팅하는 과정에서 데이터를 numpy array 그대로 넣어주지 않고 데이터셋 객체를 만들어줄것
# tf.data.Dataset객체를 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능이 제공된다고 함

BUFFER_SIZE = len(enc_train) # 전체 데이터수 보다 크거나 같은 값으로 설정해야 모두를 섞을 수 있음
BATCH_SIZE = 256 # 배치 사이즈 설정
# steps_per_epoch = len(enc_train) // BATCH_SIZE # 한 번의 epoch에 몇번의 스텝이 있는지 설정하는데 어디에 쓰이는지 모르겠음..(???)
                                               # 제외하겠음

# tokenizer에서 구축한 7000개 + 포함되지 않은 0:<pad> = 7001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듬
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# validation 데이터 또한 tf.data.Dataset객체로 만들어줌
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset.shuffle(len(enc_val))

# AI 학습

In [ ]:
# 모델을 만들어주는 클래스를 만듬
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # 임베딩 레이어는 입력 텐서의 단어 사전 인덱스 값을 이에 해당하는 워드 벡터로 바꿔주는 역할
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        # 자연어 처리에서 거의 기본적으로 쓰이는 RNN구조인 LSTMM 레이어를 두 층 추가해줌
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # 임베딩 레이어에 들어가는 인자
                     # 값이 클수록 단어의 추상적 의미를 잘 파악하지만 데이터가 충분하지 않으면 오히려 혼란의 야기함
hidden_size = 1024 # 모델을 처리하는 일꾼의 수라고 이해하면 쉬움
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[ 9.65514701e-05,  2.27478668e-04, -1.09749672e-04, ...,
         -4.77829832e-04,  1.73419085e-05, -2.80712033e-04],
        [-3.91485839e-04,  2.49289995e-04, -3.92608228e-04, ...,
         -6.97694079e-04,  1.16953843e-05, -1.56791299e-04],
        [-7.03829923e-04,  3.32247320e-04, -2.02902273e-04, ...,
         -6.65610540e-04, -6.05587957e-05,  3.52315110e-04],
        ...,
        [ 8.42815149e-04,  1.26152695e-03,  3.73328570e-03, ...,
         -3.15485318e-04, -1.91503612e-04,  4.80162562e-04],
        [ 1.09656737e-03,  1.23537204e-03,  4.11703438e-03, ...,
         -1.81989875e-04, -3.99948593e-04,  5.58939180e-04],
        [ 1.33975665e-03,  1.23028096e-03,  4.44319379e-03, ...,
         -3.05146023e-05, -5.83747402e-04,  6.22721098e-04]],

       [[ 9.65514701e-05,  2.27478668e-04, -1.09749672e-04, ...,
         -4.77829832e-04,  1.73419085e-05, -2.80712033e-04],
        [ 2.31499318e-04,  2.08288766e-04,  2.

In [ ]:
# 모델 확인
model.summary()

Model: "text_generator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  1792256   
                                                                 
 lstm_2 (LSTM)               multiple                  5246976   
                                                                 
 lstm_3 (LSTM)               multiple                  8392704   
                                                                 
 dense_1 (Dense)             multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [57]:
optimizer = tf.keras.optimizers.Adam() # 옵티마이저는 Adam으로 설정
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer) # 30번을 학습
history1 = model.fit(dataset,
                    epochs=10,
                   validation_data=dataset_val,
                   verbose=1)

Epoch 1/10
 39/562 [=>............................] - ETA: 1:09 - loss: 0.9103

KeyboardInterrupt: ignored

In [71]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence를 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듬
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [72]:
generate_text(model, tokenizer, init_sentence="<start> I was ", max_len=20)

'<start> i was playin in the <unk> , the mood all changed <end> '

- 너도 망했어야 해??

In [66]:
s1 = generate_text(model, tokenizer, init_sentence="<start> i m", max_len=20)
s2 = generate_text(model, tokenizer, init_sentence="<start> you were", max_len=20)
s3 = generate_text(model, tokenizer, init_sentence="<start> he is", max_len=20)
s4 = generate_text(model, tokenizer, init_sentence="<start> she is", max_len=20)
s5 = generate_text(model, tokenizer, init_sentence="<start> that was", max_len=20)
s6 = generate_text(model, tokenizer, init_sentence="<start> there are", max_len=20)

In [ ]:
print(s1)
print(s2)
print(s3)
print(s4)
print(s5)
print(s6)

<start> i m a negative creep <end> 
<start> you were my first love and you will be my last <end> 
<start> he is a foreign man <end> 
<start> she is not fourteen . how long is it now <end> 
<start> that was the cause of my imprisonment . <end> 
<start> there are people dying <end> 


1. 나는 부정적인 머저리다.
2. 너는 나의 첫사랑이었고 너는 나의 마지막이 될거야.
3. 그는 외부인이다.
4. 그녀는 14살이 아닙니다. 얼마나 지난거야
5. 그것이 나의 투옥의 원인이었다.
6. 죽어가는 사람들이 있다

---

노래 가사를 가져와서 학습을 시키니 뭔가 굉장히 자극적인 것이 나왔다.